Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [15]:
from random import random, seed
from itertools import product
import numpy as np
import matplotlib.pyplot as plt

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [16]:
UNIVERSE_SIZE = 10000
NUM_SETS = 1000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [17]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [18]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [19]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(True, 4275095.880046674)

In [20]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(True, 2136370.3910243707)

### Solution

#### Tweak and Restart

In this solution, the algorithm starts with an empty set of selected sets. It then randomly adds sets one by one until the solution covers all elements. The number of iterations for restarting the process is defined and, at each restart, a new random solution is generated. After each valid solution is found, the cost is compared with the current best solution. If the new solution has a lower cost, it replaces the best solution. This process of restarting helps explore different random combinations of sets.

This approach is inspired by the Hill Climber and Restart algorithms presented in the Computational Intelligence course (01URROV).

In [7]:
# Tweak and restart

# Best solution - initializes with all sets
best = np.full(NUM_SETS, True)

# How many times it should restart
max_restarts = 100

for i in range(max_restarts):

    # Initialize the solution (start with no sets selected)
    solution = np.full(NUM_SETS, False)

    # Keep adding sets until the solution is valid
    while not valid(solution):
        # Randomly pick a set that is not already in the solution
        index = np.random.randint(0, NUM_SETS)
        if not solution[index]:  # Only add if not already in the solution
            solution[index] = True
        
           

    # Return the valid solution and its cost
    #print(valid(solution), cost(solution))

    if cost(solution) < cost (best):
        best = solution.copy()


print(valid(best), cost(best))

True 145951.86525494716




#### Stronger Tweak and Restart

This approach is a stronger version of the previous one, adding randomness more dynamically. Instead of simply picking a random set to add, the algorithm introduces a probability factor using a while loop to increase the likelihood of choosing multiple sets at once. This makes the solution generation more diverse by exploring more possible combinations in each restart. 

Like the previous method, this process is inspired by the Hill Climber, Stronger Tweak and Restart algorithms presented in the Computational Intelligence course (01URROV).



In [8]:
# Stronger tweak and restart

# Best solution - initializes with all sets
best = np.full(NUM_SETS, True)

# How many times it should restart
max_restarts = 100

for i in range(max_restarts):

    # Initialize the solution (start with no sets selected)
    solution = np.full(NUM_SETS, False)

    # Keep adding sets until the solution is valid
    while not valid(solution):
        index = None
        while index is None or np.random.random() < 0.4:
            index = np.random.randint(0, NUM_SETS)
        
            if not solution[index]:  # Only add if not already in the solution
                solution[index] = True

    # Return the valid solution and its cost
    #print(valid(solution), cost(solution))

    if cost(solution) < cost (best):
        best = solution.copy()


print(valid(best), cost(best))

True 153245.97758340053


#### Greedy Algorithm

This algorithm starts with no sets selected and progressively adds sets that cover the most uncovered elements at each step. The set with the highest number of uncovered elements and the lowest cost is chosen to minimize the overall cost. The algorithm iterates through all sets, checking both coverage and cost efficiency. After selecting the best set, it updates the list of uncovered elements and continues until all elements are covered. 

The greedy algorithm implemented here is based on the description of the algorithm from the Wikipedia reference provided in the notebook.



In [9]:
# Greedy algorithm

# Initialize the solution (start with no sets selected)
solution = np.full(NUM_SETS, False)

# Array with all elements uncovered (initializes with all elements)
uncovered_elements = np.where(np.any(SETS, axis=0))[0]

while not valid(solution):

    # Cost of the best set (initializes with the cost of all sets)
    best_set_cost = cost(np.full(NUM_SETS, True))

    # Maximum number of uncovered elements in a set
    max_uncovered = 0

    # Set with more uncovered elements and lowest cost
    best_set_index = None

    for set_index in range(len(SETS)):

        # Checks if set has already been used
        if not solution[set_index]:

            # Number of elements still uncovered that a set covers
            uncovered_elements_in_set = len(np.intersect1d(uncovered_elements, np.where(SETS[set_index])[0]))
            
            # Set cost 
            set_cost = COSTS[set_index]

            # Update the best set based on uncovered elements and cost
            if uncovered_elements_in_set > max_uncovered:
                max_uncovered = uncovered_elements_in_set
                best_set_index = set_index
                best_set_cost = set_cost
            elif uncovered_elements_in_set == max_uncovered and set_cost < best_set_cost:
                best_set_index = set_index
                best_set_cost = set_cost
    
    # Adds set to solution
    solution[best_set_index] = True

    # Updates list of uncovered elements
    uncovered_elements = np.setdiff1d(uncovered_elements, np.where(SETS[best_set_index])[0])
    

print(valid(solution), cost(solution))        
        

True 99879.28670834284


#### Randomly Removes One Set at a Time

In this approach, the solution starts with all subsets selected, meaning every element is initially covered. The algorithm randomly removes one set at a time and checks if the solution remains valid. If removing the set does not compromise the solution's validity, the solution is updated to this new configuration. The algorithm repeats this process for a fixed number of removal attempts. This process aims to reduce the solution's cost by eliminating unnecessary sets while ensuring that all elements remain covered.

The inspiration for this code came from discussions on potential solutions during the Computational Intelligence course (01URROV).



In [10]:
# Randomly removes one set at a time

# Initialize the solution (start with all sets selected)
solution = np.full(NUM_SETS, True)  

# Perform the random removal process multiple times
num_removals = 100  

for _ in range(num_removals):

    new_solution = solution.copy()

    # Randomly pick a subset to remove
    set_to_remove = np.random.randint(0, NUM_SETS)
    new_solution[set_to_remove] = False

    # Check if the new solution is still valid
    if valid(new_solution):
        solution = new_solution


print(valid(solution), cost(solution))


True 3865929.870894736


#### Randomly Removes Sets

This variation of the previous approach introduces a probabilistic mechanism for removing multiple sets in a single step. Instead of removing exactly one set, the algorithm may remove several sets at once based on a 90% probability. This allows for more aggressive exploration of possible solutions. Like the single-removal version, the algorithm ensures that the new configuration is valid before updating the solution. This method offers a broader exploration of possible configurations and can potentially yield a more cost-efficient solution in fewer steps.

The inspiration for this code came from discussions on potential solutions during the Computational Intelligence course (01URROV).



In [11]:
# Randomly removes sets

# Initialize the solution (start with all sets selected)
solution = np.full(NUM_SETS, True)  

# Perform the random removal process multiple times
num_removals = 100  

for _ in range(num_removals):

    new_solution = solution.copy()

    # Randomly remove sets with a 90% chance for additional removals
    index = None
    while index is None or np.random.random() < 0.9:
        # Pick a random set to remove
        index = np.random.randint(0, NUM_SETS)

        # Only remove if it's currently in the solution 
        if new_solution[index]:
            new_solution[index] = False

    # Check if the new solution is still valid
    if valid(new_solution):
        solution = new_solution

print(valid(solution), cost(solution))


True 1407753.3226824626


### Performance Comparison

- **Tweak and Restart**: This method explores random solutions without specific strategies, relying on multiple restarts to find a better configuration. It may take longer to converge to a good solution and is less efficient than structured approaches. Its final cost might be higher as it lacks focus on cost efficiency.
  
- **Stronger Tweak and Restart**: This variation introduces a probabilistic selection process, exploring more diverse combinations. It performs slightly better than the simpler tweak and restart, as it samples more potential solutions, but it is still not as efficient as other methods in minimizing the cost.

- **Greedy Algorithm**: The greedy approach systematically selects the most cost-effective and coverage-efficient sets, leading to a much more optimized solution. It prioritizes covering as many uncovered elements as possible with the least cost, resulting in a better solution than random tweaks in both cost and performance. It converges quickly and is likely to achieve the lowest cost.

- **Randomly Removes One Set at a Time**: This method starts with the maximum coverage and gradually reduces unnecessary sets. It can perform well if there are many redundant sets, but removing only one set at a time may limit its effectiveness in finding the most optimal solution.

- **Randomly Removes Sets**: The probabilistic removal of multiple sets introduces greater flexibility in exploring different solutions. It can lead to a faster reduction in cost but might also remove necessary sets, requiring more iterations to maintain a valid solution. This method strikes a balance between exploration and solution validity but can sometimes remove sets too aggressively.

Overall, the **greedy algorithm** typically offers the best performance in terms of minimizing cost and efficiency due to its systematic approach. The **random removal strategies** can also be effective, especially when redundant sets are present, but they tend to require more iterations to converge to a low-cost solution.